In [ ]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import sys
import matplotlib.pyplot as plt

In [ ]:
# MNIST 데이터셋을 불러와 학습셋과 테스트셋으로 저장합니다.
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
# 학습셋과 테스트셋이 각각 몇 개의 이미지로 되어 있는지 확인합니다.
print("학습셋 이미지 수 : %d 개" % (X_train.shape[0]))
print("테스트셋 이미지 수 : %d 개" % (X_test.shape[0]))

In [ ]:
# 첫 번째 이미지를 확인해 봅시다.
plt.imshow(X_train[0], cmap='Greys')
plt.show()

In [ ]:
X_train[0].shape

In [ ]:
#X_train[0]

In [ ]:
for x in X_train[0]:
    for i in x:
        sys.stdout.write("%-3s" % i)
    sys.stdout.write('\n')

In [ ]:
X_train.shape

In [ ]:
X_train.shape[0]

In [ ]:
X_train = X_train.reshape(X_train.shape[0], 784)
#(60000, 28, 28) => (60000, 784)

In [ ]:
X_train.shape

In [ ]:
X_train = X_train.astype('float64')
X_train = X_train / 255

In [ ]:
X_train[0]

In [ ]:
X_test = X_test.reshape(X_test.shape[0], 784).astype('float64') / 255

In [ ]:
# 클래스 값을 확인해 봅니다.
print("class : %d " % (y_train[0]))

In [ ]:
y_train[59999]

In [ ]:

# 바이너리화 과정을 실습해 봅니다.
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

print(y_train[0])

In [ ]:
# MNIST 데이터를 불러옵니다.
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# 차원 변환 후, 테스트셋과 학습셋으로 나누어 줍니다.
X_train = X_train.reshape(X_train.shape[0], 784).astype('float32') / 255
X_test = X_test.reshape(X_test.shape[0], 784).astype('float32') / 255

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [ ]:
#DNN 딥러닝 구조로 이미지 분류 모델

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping

In [ ]:
model = Sequential()

In [ ]:
model.add(Dense(512,input_dim=28*28, activation='relu'))
#히든 계층 1~2개 더 추가
#출력 차원을 변경해가면서 성능 비교

In [ ]:
model.add(Dense(256,activation='relu'))

In [ ]:
model.add(Dense(128,activation='relu'))
model.add(Dense(32,activation='relu'))

In [ ]:
model.add(Dense(10,activation='softmax'))

In [ ]:
model.summary()

In [ ]:
# 모델 실행 환경을 설정합니다.
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
import os

In [ ]:
modelpath="MNIST_MLP.hdf5"

In [ ]:
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)

In [ ]:
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)

In [ ]:
# 모델을 실행합니다.
history = model.fit(X_train, y_train, validation_split=0.25, epochs=30, batch_size=200, verbose=0, callbacks=[early_stopping_callback,checkpointer])


In [ ]:
# 테스트 정확도를 출력합니다.
print("\n Test Accuracy: %.4f" % (model.evaluate(X_test, y_test)[1]))

In [ ]:
#CNN 딥러닝 구조로 이미지 분류 모델

In [ ]:
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D


In [ ]:
# 데이터를 불러옵니다.
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32') / 255
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32') / 255
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
X_train[0].shape

In [ ]:
# 컨볼루션 신경망의 설정
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), input_shape=(28, 28, 1), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128,  activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [ ]:
# 모델의 실행 옵션을 설정합니다.
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
modelpath="MNIST_CNN.hdf5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)


In [ ]:
# 모델을 실행합니다.
history = model.fit(X_train, y_train, validation_split=0.25, epochs=30, batch_size=200, verbose=1, callbacks=[early_stopping_callback,checkpointer])


In [ ]:
# 테스트 정확도를 출력합니다.
print("\n Test Accuracy: %.4f" % (model.evaluate(X_test, y_test)[1]))

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.datasets import reuters       # 로이터 뉴스 데이터셋 불러오기
from tensorflow.keras.callbacks import EarlyStopping

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# 데이터를 불러와 학습셋, 테스트셋으로 나눕니다.
(X_train, y_train), (X_test, y_test) = reuters.load_data(num_words=1500, test_split=0.2)
#로이터 뉴스 전체에 등장하는 단어들 중에서 가장 빈도가 높은 1500개의 단어만 사용
#          단어1 ... 단어1500
# 뉴스기사
# 1         5  0 ...   1  2
# ...
8982

#8982 * 200000=????


In [ ]:
X_train
len(X_train) #8982

In [ ]:
len(X_test) #2246

In [ ]:
y_train
len(y_train) #8982
np.unique(y_train) #0~45 : 뉴스 카테고리

In [ ]:
print(len(X_train[0])) #로이터 뉴스 기사에 등장하는 영어 단어 -> 수치 인코딩

In [ ]:
print(y_train[0]) #로이터 뉴스 기사의 종류(46가지) -> 수치 인코딩

In [ ]:
category = np.max(y_train) + 1
print(category, '카테고리')
print(len(X_train), '학습용 뉴스 기사')
print(len(X_test), '테스트용 뉴스 기사')
print(X_train[0])

# 뉴스기사
# 오늘은 날씨가 흐립니다. 오후 내내 비가 내릴 예정입니다.
# => 3(오늘), 7(날씨), 23(흐림)
# 카테고리
# 5(날씨)

In [ ]:
# LSTM 입출력 구조
# - 1:1, 1:다, 다:1, 다:다
# (여러 개 입력 단어) 다 : (분류 결과) 1
len(X_train[0]) #87
len(X_train[1]) #56

#X_train에 저장된 데이터의 최대 길이를 출력
# max(map(len, X_train)) #2376
# max_len, idx = 0, 0
# for i, k in enumerate(X_train):
#     if max_len < len(k):
#         max_len = len(k)
#         idx = i
# print(max_len, i) #2376 8981

In [ ]:
# 단어의 수를 맞추어 줍니다. zero-padding
X_train = sequence.pad_sequences(X_train, maxlen=100)
X_test = sequence.pad_sequences(X_test, maxlen=100)

In [ ]:
X_train[8981]

In [ ]:

# 원-핫 인코딩 처리를 합니다.
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
model = Sequential()
model.add(Embedding(1500, 200)) #1500차원의 단어벡터공간 -> 임베딩 -> 200차원 벡터 공간
model.add(LSTM(100, activation='tanh'))
model.add(Dense(46, activation='softmax'))

In [ ]:
# 모델의 실행 옵션을 정합니다.
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 학습의 조기 중단을 설정합니다.
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5)

# 모델을 실행합니다.
history = model.fit(X_train, y_train, batch_size=20, epochs=200, validation_data=(X_test, y_test), callbacks=[early_stopping_callback])

# 테스트 정확도를 출력합니다.
print("\n Test Accuracy: %.4f" % (model.evaluate(X_test, y_test)[1]))


















In [ ]:
!pip install kaggle --upgrade

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!ls -1ha kaggle.json

In [ ]:
!kaggle competitions download -c dogs-vs-cats

In [ ]:
!ls

In [ ]:
!unzip dogs-vs-cats.zip

In [ ]:
!unzip train.zip

In [ ]:
!unzip test1.zip

In [ ]:
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os
print(os.listdir())

In [ ]:
FAST_RUN = False
IMAGE_WIDTH=128
IMAGE_HEIGHT=128
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS=3

In [ ]:
filenames = os.listdir("/content/train")


In [ ]:
categories = []
for filename in filenames:
    category = filename.split('.')[0]
    if category == 'dog':
        categories.append(1)
    else:
        categories.append(0)

df = pd.DataFrame({
    'filename': filenames,
    'category': categories
})

In [ ]:
df.head()


In [ ]:
df['category'].value_counts().plot.bar()


In [ ]:
sample = random.choice(filenames)
image = load_img("/content/train/"+sample)
plt.imshow(image)